Here are the steps we will follow:

Generate synthetic time series data
Split the data into training and testing sets
Define the autoregressive model
Train the model using rolling window cross validation
Evaluate the performance of the model on the testing set

In [1]:
import numpy as np

np.random.seed(0)

n_obs = 500
data = np.random.normal(loc=0, scale=1, size=n_obs)


Step 2: Split the data into training and testing sets

We will split the data into a training set containing the first 400 observations and a testing set containing the remaining 100 observations.

In [ ]:
train_data = data[:400]
test_data = data[400:]

Step 3: Define the autoregressive model

We will define a simple autoregressive model that predicts the next observation in the time series based on the previous observation. Specifically, the model predicts the next observation as a linear combination of the previous observation and a random noise term.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense

def create_model():
    inputs = Input(shape=(1,))
    x = Dense(1, activation='linear')(inputs)
    model = tf.keras.models.Model(inputs=inputs, outputs=x)
    return model


Step 4: Train the model using rolling window cross validation

We will train the model using rolling window cross validation. Specifically, we will define a function that takes in the training data, the size of the training window, and the size of the step size. The function will then loop over the training data, training the model on a window of data and making predictions on the next observation. We will save the predictions and the actual values for each testing window.

In [ ]:
def rolling_window_cross_validation(train_data, window_size, step_size):
    model = create_model()
    model.compile(optimizer='adam', loss='mean_squared_error')

    n_windows = int((len(train_data) - window_size) / step_size) + 1
    predictions = []
    actuals = []

    for i in range(n_windows):
        start = i * step_size
        end = start + window_size
        train_window = train_data[start:end]
        test_window = train_data[end:end+1]

        for j in range(len(train_window)):
            x_train = train_window[j:j+1]
            y_train = train_window[j+1:j+2]
            model.fit(x_train, y_train, epochs=1, verbose=0)

        x_test = test_window[:-1]
        y_test = test_window[-1:]
        y_pred = model.predict(x_test)
        predictions.append(y_pred)
        actuals.append(y_test)

    return predictions, actuals


Step 5: Evaluate the performance of the model on the testing set

We will use the predictions and actual values generated from the rolling window cross validation to evaluate the performance of the model on the testing set. Specifically, we will calculate the mean squared error (MSE) between the predicted and actual values.

In [ ]:
window_size = 20
step_size = 10

predictions, actual =rolling_window_cross_validation(train_data, window_size, step_size)